# Biredctional LSTM

In [ ]:
import json
import numpy as np
import re
import io
import nltk
import h5py
import keras as k
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import concatenate
from sklearn.metrics import f1_score

CREATING EMBEDING LAYER WITH HELP OF WORD EMBEDDINGS: GLOVE

In [4]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


IMPORT VECTORIZED FILES FOR CONTEXT, QUESTIONS, ANSWERS POST PREPROCESS OF DATA

In [5]:
context = h5py.File('context.h5','r')
questions = h5py.File('questions.h5','r')
answers = h5py.File('answers.h5','r')
ans_begin = h5py.File('begin.h5','r')
ans_end = h5py.File('end.h5','r')

In [6]:
c_data = context['context'][:]
qn_data = questions['questions'][:]
ans_data = answers['answers'][:]

In [7]:
begin_ans = ans_begin['begin'][:]
end_ans = ans_end['end'][:]

In [8]:
# loding vocabulary
word_index = np.load('words.npy').item()

In [9]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
vocab_size = len(word_index) + 1
#embedding_vector_length = 50
batch = 64
max_span_begin = np.amax(begin_ans)
max_span_end = np.amax(end_ans)
slce = 10000


In [11]:
print("Vocab Size")
vocab_size

Vocab Size


119616

MODEL CREATION

In [12]:
context_input = Input(shape=(700, ), dtype='int32', name='c_data')
x = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=700, trainable=False)(context_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(x)
drop_1 = Dropout(0.5)(lstm_out)

In [13]:
ques_input = Input(shape=(100, ), dtype='int32', name='qn_data')
x = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=100, trainable=False)(ques_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(x)
drop_2 = Dropout(0.5)(lstm_out)

In [14]:
merge_layer = concatenate([drop_1, drop_2], axis=1)
biLSTM = Bidirectional(LSTM(512, implementation=2), merge_mode='mul')(merge_layer)
drop_3 =  Dropout(0.5)(biLSTM)
softmax_1 = Dense(max_span_begin, activation='softmax')(biLSTM)
softmax_2 = Dense(max_span_end, activation='softmax')(biLSTM)
model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c_data (InputLayer)             (None, 700)          0                                            
__________________________________________________________________________________________________
qn_data (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 700, 100)     11961600    c_data[0][0]                     
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 100)     11961600    qn_data[0][0]                    
__________________________________________________________________________________________________
bidirectio

In [72]:
model_history = model.fit([c_data[:slce], qn_data[:slce]],
                        [begin_ans[:slce], end_ans[:slce]], verbose=2,
                         batch_size=batch, epochs=50)

Epoch 1/50
 - 713s - loss: 11.9526 - dense_1_loss: 5.9232 - dense_2_loss: 6.0295 - dense_1_acc: 0.0438 - dense_2_acc: 0.0220
Epoch 2/50
 - 713s - loss: 11.7656 - dense_1_loss: 5.8307 - dense_2_loss: 5.9348 - dense_1_acc: 0.0462 - dense_2_acc: 0.0247
Epoch 3/50
 - 712s - loss: 11.5827 - dense_1_loss: 5.7407 - dense_2_loss: 5.8420 - dense_1_acc: 0.0480 - dense_2_acc: 0.0303
Epoch 4/50
 - 712s - loss: 11.4807 - dense_1_loss: 5.6900 - dense_2_loss: 5.7907 - dense_1_acc: 0.0519 - dense_2_acc: 0.0323
Epoch 5/50
 - 712s - loss: 11.3027 - dense_1_loss: 5.6021 - dense_2_loss: 5.7006 - dense_1_acc: 0.0547 - dense_2_acc: 0.0375
Epoch 6/50
 - 713s - loss: 11.1123 - dense_1_loss: 5.5071 - dense_2_loss: 5.6053 - dense_1_acc: 0.0599 - dense_2_acc: 0.0445
Epoch 7/50
 - 712s - loss: 10.9393 - dense_1_loss: 5.4209 - dense_2_loss: 5.5183 - dense_1_acc: 0.0620 - dense_2_acc: 0.0505
Epoch 8/50
 - 712s - loss: 10.7678 - dense_1_loss: 5.3370 - dense_2_loss: 5.4308 - dense_1_acc: 0.0664 - dense_2_acc: 0.0551


IMPORT VALIDATION DATA POST PREPROCESSING FOR TESTING

In [73]:
t_context = h5py.File('context_test.h5','r')
t_questions = h5py.File('questions_test.h5','r')
t_answers = h5py.File('answers_test.h5','r')
t_ans_begin = h5py.File('begin_test.h5','r')
t_ans_end = h5py.File('end_test.h5','r')

In [74]:
t_c_data = t_context['context'][:]
t_qn_data = t_questions['questions'][:]
t_ans_data = t_answers['answers'][:]
t_begin_ans = t_ans_begin['begin'][:]
t_end_ans = t_ans_end['end'][:]

In [75]:
predictions = model.predict([t_c_data,t_qn_data], batch_size=128)

In [76]:
print(predictions[0].shape, predictions[1].shape)

(20302, 3126) (20302, 3136)


MODEL PERFORMANCE EVALUATION:
    METRICS USED:
        1. F1 SCORE
        2. EXACT MATCH SCORE

In [77]:
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 

In [78]:
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()
print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())

0 1373 3 1211


F1 Score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0

In [79]:
f1_b = f1_score(t_begin_ans, ansBegin, average='micro')  

In [80]:
f1_e = f1_score(t_end_ans, ansEnd, average='micro') 

In [81]:
print("F1 Score")
f1_b + f1_e

F1 Score


0.007437690867894788

Exact Match Score: One way of calculating it is through Accuracy classification score.
It computes subset accuracy: the set of answers predicted for a sample data must exactly match the corresponding set of labels in true answers.

In [82]:
em_b = accuracy_score(t_begin_ans, ansBegin)

In [83]:
em_e = accuracy_score(t_end_ans, ansEnd)

In [84]:
print("Exact Match Score")
em_b + em_e

Exact Match Score


0.007437690867894788